In [5]:
%pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


In [143]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from datetime import date,timedelta,datetime
from dateutil.parser import parse
from selenium.webdriver.common.action_chains import ActionChains
import dill as pickle

In [3]:
# Daily Discission Threads
# Turns off notifications for our driver so our scrape doesn't break at any point
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)


In [140]:
#Functions
def select_sort_filter(filtr:str = "New"):
    """
    Args: filtr
    Select how you want the results to be filtered.
    Options are New, Hot, Top, Relevance, and Most Comments
    "New" by default
    """
    filter_select_button = driver.find_element_by_xpath('//button[@data-testid = "search-results-filter-sort"]')
    filter_select_button.click()
    
    filter_button = driver.find_element_by_xpath(f'//button[@role = "menuitem"]//span[text()="{filtr}"]')
    filter_button.click()
def pull_Comments(filtr_comments: str = "top"):
    """
    Args: filtr_comments
    Select how you want post comments to be filtered.
    Options are best, top, new, controversial, old, or q&a
    Returns:
    List of comments on the selected post
    This includes the comment_id, comment_author, text, and number of upvotes
    """
    comments = []
    # Tries to click the comment drop down menu
    try:
        comments_sort = driver.find_element(By.XPATH,('//span[@class="_2-cXnP74241WI7fpcpfPmg _3LRBCA71BwLLXBNsSlY7HW"]'))
        driver.execute_script("arguments[0].click();", comments_sort)
        time.sleep(3)
    except:
        print("sort dropdown error")
    #tries to click the desired sort value from arg:filtr_comments
    try:
        comments_sort_select = driver.find_element(By.XPATH,(f'//button[@class="_10K5i7NW6qcm-UoCtpB3aK _3LwUIE7yX7CZQKmD2L87vf _1oYEKCssGFjqxQ9jJMNj5G"]//span[text() ="{filtr_comments}"]'))
        driver.execute_script("arguments[0].click();", comments_sort_select)
        time.sleep(2)
    except:
        print("sort select error")
        
    #pull the value displayed for sorting on the website and checks if it matches our desired sort
    sorted_ = driver.find_element(By.XPATH,('//span[@class="_2-cXnP74241WI7fpcpfPmg _3LRBCA71BwLLXBNsSlY7HW"]'))
    sorted_by = "".join(sorted_.text.split(' ')[-1:])
    
    if sorted_by.lower() != filtr_comments:
            print('comments not properly sorted')
    
    # we are done with buttons so we create a soup and pull all comment objects
    soup = BeautifulSoup(driver.page_source)
    comment_cells = soup.find('div', class_ = "_1YCqQVO-9r-Up6QPB9H6_4 _1YCqQVO-9r-Up6QPB9H6_4").find_all('div', class_ = "_3sf33-9rVAO_v4y0pIW_CH")
    
    # Loop through all comment objects
    for ix, cell in enumerate(comment_cells):
        # Checks for a username to ensure this is a proper comment
        if cell.find('div', class_= "_2mHuuvyV9doV3zwbZPtIPG"):
            
            comment = {}
            comment_id = cell.get('id')
            
            # Checks that this is not a "x more replies" option
            if 'moreComments' not in comment_id:
                
                comment.update({'comment_id':comment_id})
                #takes the username from the comment
                username = cell.find('div', class_= "_2mHuuvyV9doV3zwbZPtIPG").find('a')
                comment.update({'username':username.get('href')})
                
                #comments are classed line by line, this finds them all and pastes them together
                comment_body =cell.find_all('p')
                line_body = ""
                for line in comment_body:
                    line_body = line_body + " " + line.text
                
                comment.update({'body':line_body})
                
                #gets the number of upvotes for that comment, if there are any at all
                comment_upvotes = cell.find('div', class_ = "_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ _3ChHiOyYyUkpZ_Nm3ZyM2M")
                if comment_upvotes:
                    comment.update({"upvotes":comment_upvotes.text})

                comments.append(comment)
            
    return comments    
    
def pull_Daily_Discussion(filtr_comments: str = "top"):
    """
    Args: filtr_comments
    Select how you want post comments to be filtered.
    Options are best, top, new, controversial, old, or q&a
    Returns: 
    List of posts from the specified url "Daily Discussion"
    """
    # This worked when I last tried it, but I didn't add any of the bug fixes that I did to pull_Posts
    # Plus I think pull_Posts works on daily discussion as well
    posts = []
    
    links = driver.find_elements(By.XPATH,('//h3'))
    
    for ix,link in enumerate(links):
        post = {}
        post.update({'title':link.text}) 
        date = "".join(link.text.split(' ')[-3:])
        post.update({"Reference_Date":parse(date)})
        
        link.click()
        time.sleep(5)
        
        post_body = driver.find_element(By.XPATH,('//p[@class="_1qeIAgB0cPwnLhDF9XSiJM"]'))
        post.update({'body':post_body.text})
        time.sleep(1)
        
        num_comments = driver.find_element(By.XPATH,('//span[@class="FHCV02u6Cp2zYL0fhQPsO"]'))
        post.update({'num_comments': num_comments.text})
        time.sleep(2)
        nums_upvotes_rev = driver.find_elements(By.XPATH,('//span[@class="WBY6J5DJsZFZXSxBqtq0F"]//span'))
        if nums_upvotes_rev:
            nums_upvotes = [num.text for num in reversed(nums_upvotes_rev)]
            
            num_upvotes = ''.join(nums_upvotes)
            post.update({'num_upvotes':num_upvotes})
        else:
            num_upvotes = driver.find_element(By.XPATH,('//div[@class="_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ"]'))
            post.update({'num_upvotes':num_upvotes.text})
        #time.sleep(5)
        
        post_author = driver.find_element(By.XPATH,('//a[@class="_2tbHP6ZydRpjI44J3syuqC  _23wugcdiaj44hdfugIAlnX oQctV4n0yUb0uiHDdGnmE"]'))
        post.update({'author':post_author.text})
        
        comments = pull_Comments(filtr_comments)
        
        post.update({'comments':comments})
        
        close_button = driver.find_element(By.XPATH,('//button[@title = "Close"]'))
        close_button.click()
        close_button.click()
        
        posts.append(post)
        
        time.sleep(5)
        
    return(posts)

def pull_Posts(filtr_comments: str = "top"):
    """
    Args: filtr_comments
    Select how you want post comments to be filtered.
    Options are best, top, new, controversial, old, or q&a
    Returns: 
    List of posts from the specified url
    """
    
    posts = []
    
    # First part here scrolls to the bottom of the page to load more posts
    # This repeats until there are 100 posts, or until it has scrolled 100 times
    # Finishes by returning to the top
    i=0
    page_length = []
    while len(page_length) <= 100:
        i += 1
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
        page_length = driver.find_elements(By.XPATH,('//div[@class = "_14-YvdFiW5iVvfe5wdgmET"]'))
        if i == 100:
            break
    driver.execute_script("window.scrollTo(0,0)")
    time.sleep(10)
    
    # pulls all the posts that have just been loaded
    links = driver.find_elements(By.XPATH,
                                 ('//div[@class = "_1poyrkZ7g36PawDueRza-J _11R7M_VOgKO1RJyRSRErT3 "]'))
    # for each post
    for ix,link in enumerate(links):
        post = {}
        # this field only exists on promoted posts, we don't want to click those
        is_promoted = link.find_elements(By.XPATH, 
                                         (".//span[@class = '_2oEYZXchPfHwcf9mTMGMg8 V0WjfoF5BV7_qbExmbmeR']"))
        if not is_promoted:
            action = ActionChains(driver)
            # If the post title or link doesn't exist the loop is stopped
            try:
                post_link = link.find_element(By.XPATH,('.//h3'))
            except: 
                print(f'no link at elem {ix}')
                break
            try:
                post.update({'title':post_link.text})
            except:
                print(f'no text at elem {ix}')
                break
            # clicks the link to open the post
            action.move_to_element(post_link).click().perform()
            time.sleep(2)
            
            # Pulls the data from the post into soup
            soup_2 = BeautifulSoup(driver.page_source)
            # if the structure isn't what we expect for the post, we clicked something wrong and the loop
            # is broken
            try:
                date_elem = soup_2.find(
                    "div", class_ = "u35lf2ynn4jHsVUwPmNU Dx3UxiK86VcfkFQVHNXNi").find(
                "a", class_ = "_3jOxDPIQ0KaOWpzvSQo-1s") 
            except:
                print(f'finished at elem {ix}')
                break
            # to get the datetime of the post, we parse the "_ hours ago" field of the post and
            # subtract it from the current time
            date_num = int(date_elem.text.split(' ')[0])
            time_p = date_elem.text.split(' ')[1]
            if time_p == 'hours' or time_p == 'hour':
                time_d = timedelta(hours = date_num)
                date_dt = datetime.now() - time_d
            elif time_p == 'minutes' or time_p == 'minute':
                time_d = timedelta(minutes = date_num)
                date_dt = datetime.now() - time_d
            elif time_p == 'seconds' or time_p == 'seconds':
                time_d = timedelta(seconds = date_num)
                date_dt = datetime.now() - time_d
            elif time_p == 'day' or time_p == 'days':
                time_d = timedelta(seconds = date_num)
                date_dt = datetime.now() - time_d
            else:
                print (f"unkown time delta {time_p}")
            post.update({"Reference_Date":date_dt})
            
            # pulls the body of the post
            post_body = soup_2.find('h1', class_ = "_eYtD2XCVieq6emjKBH3m")
            post.update({'body':post_body.text})
            time.sleep(1)
            
            # takes the number of comments on the post
            num_comments = soup_2.find('span', class_ = "FHCV02u6Cp2zYL0fhQPsO")
            post.update({'num_comments': num_comments.text})
            time.sleep(1)
            
            # takes the tag of the post
            post_tags = soup_2.find(
                "div", class_ = "u35lf2ynn4jHsVUwPmNU Dx3UxiK86VcfkFQVHNXNi").find(
            "div", class_ = "_2fiIRtMpITeCAzXc4cANKp _1mK-LVHGTTlcFpMsjItjYJ").find(
            'span')
            post.update({'tag':post_tags.text})
            
            #takes the number of upvotes on the post
            num_upvotes = soup_2.find("div", class_ ="_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ")
            post.update({'num_upvotes':num_upvotes.text})
            
            #takes the author of the post
            post_author = driver.find_element(By.XPATH,('//a[@class="_2tbHP6ZydRpjI44J3syuqC  _23wugcdiaj44hdfugIAlnX oQctV4n0yUb0uiHDdGnmE"]'))
            post.update({'author':post_author.text})
            
            #run the function for pulling comments
            comments = pull_Comments(filtr_comments)

            post.update({'comments':comments})
            #closes the post (we need to click twice for this)
            close_button = driver.find_element(By.XPATH,('//button[@title = "Close"]'))
            close_button.click()
            close_button.click()
            #update our list
            posts.append(post)

            time.sleep(5)
            
    posts_dict = {'main': posts}        
    return(posts_dict)

In [ ]:
global driver
url = "https://www.reddit.com/r/wallstreetbets/search?q=flair_name%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new"
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(url)
select_sort_filter("Hot")
time.sleep(10)
posts_list = pull_Daily_Discussion()
driver.quit()

In [ ]:
url = "https://www.reddit.com/r/wallstreetbets/"
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(url)
time.sleep(1)
posts_dict = pull_Posts()
driver.quit()

In [148]:
with open('posts_hot_ctop_2_16_22.pkl', 'wb') as file:
    pickle.dump(posts_dict,file)

In [150]:
with open('posts_hot_ctop_2_16_22.pkl', 'rb') as file:
    test = pickle.load(file)
test['main'][0]

{'title': 'What Are Your Moves Tomorrow, February 17, 2022',
 'Reference_Date': datetime.datetime(2022, 2, 16, 16, 17, 21, 799255),
 'body': 'What Are Your Moves Tomorrow, February 17, 2022',
 'num_comments': '12.1k comments',
 'tag': 'Daily Discussion',
 'num_upvotes': '292',
 'author': 'u/AutoModerator',
 'comments': [{'comment_id': 't1_hx84k5q',
   'username': '/user/SomeDumbassSays/',
   'body': ' Silly me for investing in calls on a company that manufactures very very high end GPUs and other essential computer components.  I should’ve bought calls in checks notes the food delivery company that delivers the wrong food, cold, to the wrong address',
   'upvotes': '125'},
  {'comment_id': 't1_hx84oxj',
   'username': '/user/NrdRage/',
   'body': " Don't forget the previously bankrupted electric car that's going to a rental model and missed top and bottom and has no path to profitability.",
   'upvotes': '24'},
  {'comment_id': 't1_hx8fxvn',
   'username': '/user/Jackol4ntrn/',
   'bod